Expected Shot Aggregates for each Player

- Player shots need to be aggregated for a sequence or possession.
- Is there a maximum score of 6 for each possession?
- What happens when someone scores a behind? Does the possession end?
    - Define a possession as the whole time between each goal? Or until the opponents get the ball?


In [ ]:
import pandas as pd
import numpy as np
import warnings

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 500)

Data

In [ ]:
shots = pd.read_csv(r"/Users/ciaran/Documents/Projects/AFL/git-repositories/expected-score-model/data/predictions/shots_xs_catboost.csv")

In [ ]:
shots['Year'] = shots['Match_ID'].apply(lambda x: int(x.split("_")[1]))
shots['Opponent'] = np.where(shots['Team'] == shots['Home_Team'], shots['Away_Team'], shots['Home_Team'])

In [ ]:
shots['time_since_prev_shot'] = shots['Period_Duration'] - shots['Period_Duration'].shift(1)
shots['prev_result'] = shots['result'].shift(1)
shots['prev_team'] = shots['Team'].shift(1)
shots['prev_xscore'] = shots['xscore'].shift(1)
shots['prev_goal_probas'] = shots['goal_probas'].shift(1)
shots['prev_behind_probas'] = shots['behind_probas'].shift(1)

In [ ]:
shots.head()

In [ ]:
shots['score'].sum(), shots['xscore'].sum()

Trying to do conditional shots xscore

In [ ]:
prev_result_behind = (shots['prev_result'] == 'behind')
same_team_shot = (shots['prev_team'] == shots['Team'])
prev_shot_time_seconds = 120
short_time_since_prev_shot = (shots['time_since_prev_shot'] < prev_shot_time_seconds)

shots['team_xscore'] = np.where(prev_result_behind & same_team_shot & short_time_since_prev_shot, shots['prev_behind_probas']*shots['xscore'], shots['xscore'])

In [ ]:
shots[['score', 'xscore', 'team_xscore']].sum()

Team Aggregations

In [ ]:
def create_team_aggregation(shots, group: list):
    
    team_shots_groupby = shots.groupby(group).agg(
        score_sum = ('score', 'sum'),
        xscore_sum = ('xscore', 'sum'),
        num_shots = ('score', 'size'),
        num_games = ('Match_ID', 'nunique')
    )
    team_shots_groupby['score_per_shot'] = team_shots_groupby['score_sum'] / team_shots_groupby['num_shots']
    team_shots_groupby['score_per_game'] = team_shots_groupby['score_sum'] / team_shots_groupby['num_games']

    team_shots_groupby['xscore_per_shot'] = team_shots_groupby['xscore_sum'] / team_shots_groupby['num_shots']
    team_shots_groupby['xscore_per_game'] = team_shots_groupby['xscore_sum'] / team_shots_groupby['num_games'] 
    
    team_shots_groupby['shots_per_game'] = team_shots_groupby['num_shots'] / team_shots_groupby['num_games']   
    
    return team_shots_groupby

Expected Scores Scored

In [ ]:
xscore_year_groupby = create_team_aggregation(shots, group=['Year'])
xscore_year_groupby

In [ ]:
xscore_team_groupby = create_team_aggregation(shots, group=['Year', 'Team'])
xscore_team_groupby

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
xscore_team_groupby.describe()

In [ ]:
fig, ax = plt.subplots()

ax.scatter(x = xscore_team_groupby['xscore_per_shot'], y = xscore_team_groupby['num_shots'])

Expected Scores Conceded

In [ ]:
xconcede_team_groupby = create_team_aggregation(shots, group=['Year', 'Opponent'])
xconcede_team_groupby

In [ ]:
xconcede_team_groupby.describe()

In [ ]:
fig, ax = plt.subplots()

ax.scatter(x = xconcede_team_groupby['xscore_per_shot'], y = xconcede_team_groupby['num_shots'])

Distribution of Scores v Concedes

In [ ]:
import seaborn as sns

In [ ]:
sns.kdeplot(xscore_team_groupby['xscore_per_game'], fill=True, label='For')
sns.kdeplot(xconcede_team_groupby['xscore_per_game'], fill=True, label = 'Against')

Team Expected Score Differences Plot

In [ ]:
year = 2021
xscore_team_groupby_year = xscore_team_groupby.loc[year].sort_values(by = 'xscore_per_game')
xconcede_team_groupby_year = xconcede_team_groupby.loc[year].loc[xscore_team_groupby_year.index]

In [ ]:
def set_colours_axes_ax(ax, facecolor, spine_colour, tick_colour, label_colour):
    ax.set_facecolor(facecolor)
    ax.spines[['top', 'right']].set_visible(False)
    ax.spines[["left", "bottom"]].set_color(spine_colour)
    ax.tick_params(color=tick_colour, length=5, which="major", labelsize=6, labelcolor=label_colour)

    return ax

def plot_team_xscore_difference_rank_ax(ax, xscore_team_groupby_year, xconcede_team_groupby_year):
    
    y = xscore_team_groupby_year.index

    for i, team in enumerate(y):
        
        if xscore_team_groupby_year.iloc[i]['xscore_per_game'] > xconcede_team_groupby_year.iloc[i]['xscore_per_game']:
            linecolor = 'green'
        else:
            linecolor = 'red'
        ax.scatter(x=xscore_team_groupby_year.iloc[i]['xscore_per_game'], y=y[i], c=team_colours[team]['positive'], ec='w')
        ax.scatter(x=xconcede_team_groupby_year.iloc[i]['xscore_per_game'], y=y[i], c=team_colours[team]['positive'], ec='w')
        ax.hlines(i, xmin = xconcede_team_groupby_year.iloc[i]['xscore_per_game'], xmax=xscore_team_groupby_year.iloc[i]['xscore_per_game'], color=linecolor, linestyle='--', linewidth=1, zorder=-1)

    ax = set_colours_axes_ax(ax, '#121212', 'white', 'white', 'white')
    
    return ax

In [ ]:
fig, ax = plt.subplots()
fig.set_facecolor('#121212')
year = 2024
xscore_team_groupby_year = xscore_team_groupby.loc[year].sort_values(by = 'xscore_per_game')
xconcede_team_groupby_year = xconcede_team_groupby.loc[year].loc[xscore_team_groupby_year.index]
ax = plot_team_xscore_difference_rank_ax(ax, xscore_team_groupby_year, xconcede_team_groupby_year)

In [ ]:
xscore_team_groupby.swaplevel().loc['Brisbane']

Team Specific Expected Score Differences

In [ ]:
from highlight_text import ax_text

In [ ]:
team = 'Brisbane'
xscore_team_groupby_team = xscore_team_groupby.swaplevel().loc['Brisbane'].sort_values(by = 'xscore_per_game')
xconcede_team_groupby_team = xconcede_team_groupby.swaplevel().loc['Brisbane'].loc[xscore_team_groupby_team.index]

In [ ]:
fig, ax = plt.subplots()

x = list(xconcede_team_groupby_team.index)
ax.scatter(y=xscore_team_groupby_team['xscore_per_game'], x=x)
ax.scatter(y=xconcede_team_groupby_team['xscore_per_game'], x=x)

ax.set_xticks(x)

Rolling Expected Score Difference Plots

In [ ]:
shots['Round'] = shots['Match_ID'].apply(lambda x: x.split("_")[2])
shots['Round_ID'] = shots['Match_ID'].apply(lambda x: x.split("_")[1] + x.split("_")[2])

In [ ]:
# TODO Rename this here and in `craete_rolling_team_expected_score_groupby`
def _extracted_from_create_rolling_team_expected_score_groupby_3(shots, team, team_opp):
    team_shots = shots[shots[team_opp] == team]
    
    if team_opp == 'Team':
        result = team_shots.groupby('Round_ID')[['score', 'xscore']].sum().rename(columns = {
            'score': 'score_for',
            'xscore':'xscore_for'
        })

    else:
        result = team_shots.groupby('Round_ID')[['score', 'xscore']].sum().rename(columns = {
            'score': 'score_against',
            'xscore':'xscore_against'
        })
        result.columns = ['score_against', 'xscore_against']

    return result

def create_rolling_team_expected_score_groupby(shots, team, rolling_window = 5):
    
    team_shots_groupby = (
        _extracted_from_create_rolling_team_expected_score_groupby_3(
            shots, team, 'Team'
            )
    )
    opp_shots_groupby = (
        _extracted_from_create_rolling_team_expected_score_groupby_3(
            shots, team, 'Opponent'
        )
    )
    rolling = team_shots_groupby.merge(opp_shots_groupby, left_index=True, right_index=True)

    for col in rolling.columns:
        rolling[f'{col}_rolling'] = (
            rolling[col].rolling(window=rolling_window, min_periods=0).mean()
        )
        
    rolling['diff_rolling'] = rolling['score_for_rolling'] - rolling['score_against_rolling']
    rolling['xdiff_rolling'] = rolling['xscore_for_rolling'] - rolling['xscore_against_rolling']

    for col in ['xscore_for', 'xscore_against']:
        x = np.arange(len(rolling[col]))
        trendline = np.polyfit(x, rolling[col], 1)
        rolling[f'{col}_trend'] = np.polyval(trendline, x)
    
    return rolling

Subplots for each team

In [ ]:
from expected_score_model.visualisation.afl_colours import team_colours

In [ ]:
import os
import matplotlib.font_manager as fm


In [ ]:
def load_fonts(font_path):
    for x in os.listdir(font_path):
        if x != ".DS_Store":
            for y in os.listdir(f"{font_path}/{x}"):
                if y.split(".")[-1] == "ttf":
                    fm.fontManager.addfont(f"{font_path}/{x}/{y}")
                    try:
                        fm.FontProperties(weight=y.split("-")[-1].split(".")[0].lower(), fname=y.split("-")[0])
                    except Exception:
                        continue
font_path = r"/Users/ciaran/Documents/Projects/AFL/git-repositories/expected-score-model/notebooks/visualisations/viz/fonts"
load_fonts(font_path)

In [ ]:
import matplotlib.ticker as ticker

In [ ]:
def set_colours_axes_ax(ax, facecolor, spine_colour, tick_colour, label_colour):
    ax.set_facecolor(facecolor)
    ax.spines[['top', 'right']].set_visible(False)
    ax.spines[["left", "bottom"]].set_color(spine_colour)
    ax.tick_params(color=tick_colour, length=5, which="major", labelsize=6, labelcolor=label_colour)
    ax.xaxis.set_major_locator(ticker.MultipleLocator(10))
    ax.yaxis.set_major_locator(ticker.MultipleLocator(10))
    ax.set_ylim(0, 125)

    return ax

def get_team_colours(team):
    return team_colours[team]['positive'], team_colours[team]['negative']

def plot_rolling_lines_ax(ax, data, team, color_for, color_against):
    line_for = ax.plot(data.index, data['xscore_for_rolling'], label = 'xscore_for_rolling', color = color_for, lw=1.5)
    line_against = ax.plot(data.index, data['xscore_against_rolling'], label = 'xscore_against_rolling', color= color_against, lw=1.5)
    return ax

def plot_trend_lines_ax(ax, data, team, color_for, color_against):
    line_for = ax.plot(data.index, data['xscore_for_trend'], label = 'xscore_for_trend', color = color_for, lw=1.5, alpha = 0.5, ls = '--')
    line_against = ax.plot(data.index, data['xscore_against_trend'], label = 'xscore_against_trend', color= color_against, lw=1.5, ls='--', alpha = 0.5)
    return ax

def plot_fill_between_ax(ax, data, team, color_for, color_against):
    ax.fill_between(data.index, data['xscore_against_rolling'], data['xscore_for_rolling'], where = data['xscore_for_rolling'] > data['xscore_against_rolling'], interpolate=True, alpha=0.5, zorder=3, color=color_for)
    ax.fill_between(data.index, data['xscore_against_rolling'], data['xscore_for_rolling'], where = data['xscore_against_rolling'] >= data['xscore_for_rolling'], interpolate=True, alpha=0.5, zorder=3, color=color_against)
    return ax

def plot_lines_ax(ax, data, team, color_for, color_against, fill = False, trend = False):
    
    ax = plot_rolling_lines_ax(ax, data, team, color_for, color_against)

    if trend:
        ax = plot_trend_lines_ax(ax, data, team, color_for, color_against)

    if fill:
        ax = plot_fill_between_ax(ax, data, team, color_for, color_against)
        
    return ax

def get_latest_rolling_values(data):
    for_number = data['xscore_for_rolling'].iloc[-1]
    against_number = data['xscore_against_rolling'].iloc[-1]
    return for_number, against_number

def plot_team_rolling_ax(ax, data, team, fill = False, trend = False):
    
    color_for, color_against = get_team_colours(team)
    
    text_colour_for = "black" if color_for == "white" else "white"
    text_colour_against = "black" if color_against == "white" else "white"

    ax = set_colours_axes_ax(ax, '#121212', 'white', 'white', 'white')
    ax = plot_lines_ax(ax, data, team, fill = fill, trend = trend, color_for = color_for, color_against = color_against)
    for_number, against_number = get_latest_rolling_values(data)
    
    ax_text(
        x=0,
        y=140,
        s=f'<{team}>\n<xscore for: {for_number:.1f}>  <xscore against: {against_number:.1f}>  <avg. last 5 games>',
        highlight_textprops=[
            {'color': 'white', 'weight': 'bold', 'font': 'DM Sans'},
            {
                'size': '10',
                'bbox': {
                    'edgecolor': color_for,
                    'facecolor': color_for,
                    'pad': 1,
                },
                'color': text_colour_for,
            },
            {
                'size': '10',
                'bbox': {
                    'edgecolor': color_against,
                    'facecolor': color_against,
                    'pad': 1,
                },
                'color': text_colour_against,
            },
            {
                'size': '10',
                'bbox': {'edgecolor': 'black', 'facecolor': 'grey', 'pad': 1},
                'color': 'white',
            },
        ],
        font="Karla",
        ha="left",
        size=14,
    )

    return ax

In [ ]:
def plot_team_rolling_averages(shots):
    
    fig = plt.figure(figsize=(18, 24), dpi=300)
    fig.set_facecolor('#121212')

    nrows, ncols = 6, 3
    gspec = gridspec.GridSpec(
        ncols=ncols, nrows=nrows, figure=fig,
        hspace=0.3
    )

    team_list = shots['Team'].unique()

    for plot_counter, (row, col) in enumerate(itertools.product(range(nrows), range(ncols))):
        team = team_list[plot_counter]
        ax = plt.subplot(gspec[row, col])
        rolling = create_rolling_team_expected_score_groupby(shots, team = team, rolling_window=10)
        ax = plot_team_rolling_ax(ax, rolling, team = team, fill = True, trend = False)

    fig_text(
        x=0.13, y=0.92,
        s = "xscore 10-game rolling average.",
        size = 22,
        font = "Karla",
        color = 'white'
    )
    
    return fig, ax

In [ ]:
fig, ax = plot_team_rolling_averages(shots)

Expected Score Storytelling

Diamond Scatter Plot / Scatter Plot